# Projet numérique - Calcul différentiel II - Damien Capéraa & Camille Klopocki

## Idée 1 : La méthode des petits carrés 

### Description

**Principe :** Connaissant un point de la ligne de niveau, on trace autour de ce point un carré de côté `2*delta`. On exécute des fonctions de type find_seed sur chacun des quatre bords du carré.\
Afin de toujours avancer dans le bon sens, on met en place un système de mémorisation du côté sur lequel le nouveau point a été trouvé. L'objectif est que le point suivant regarde ce côté-ci en dernier. En effet, c'est sur celui-ci que se trouve le point précédent, le risque serait donc de retrouver ce point-là et de pas avancer... C'est le rôle de la varibale `numcôté` dans nos programmes.

**Contraintes :** Dans `contour_simple`, on regarde une fonction sur le carré [0, 1]². Pour s'assurer de ne pas sortir de ce carré-là, on ajoute des fonctions spéciales qui permettent de :

1) Compter le nombre de côtés dont le point courant est trop proche (ie à une distance inférieure à delta). 
- Dans un coin, cette variable vaut 2. 
- Près d'un côté mais hors d'un coin, elle vaut 1. 
- Ailleurs, elle vaut 0

2) Ajuster les dimensions du "carré" (qui pourra devenir un rectangle...). Si par exemple notre point courant est très proche du bord gauche de notre carré [0, 1]², on déplace le bord gauche de notre petit carré sur la ligne verticale d'abscisse x=0. Ainsi, si le point suivant est trouvé sur ce bord, il sera toujours dans le carré [0, 1]².

**Conditions d'arrêt :** Toujours dans `contour_simple`, nous avons pris comme condition d'arrêt :
- l'arrivée sur un bord.
- le bouclage (la ligne de niveau a fait un cercle). Il est cependant peu probable que cette condition soit appliquée dans le programme final, car le quadrillage de la zone de définition de notre fonction devrait être suffisamment fin pour empêcher la ligne de niveau de boucler sur de si petites surfaces.

### Place au code

#### La fonction find_seed

On crée également find_seed_y0, dans laquelle c'est y qui est fixé à y0 et x qui varie entre valminx et valmaxx. On en aura besoin lors de notre recherche du point suivant sur les quatre bords du petit carré (y compris les bords horizontaux).

#### La fonction contour_simple

Une fois notre premier point trouvé sur le bord gauche, il s'agit de savoir si ce point est dans un coin (ie à une distance inférieur à delta de deux bords) ou pas. 
`test_bord` renvoie ainsi une liste `problemes` de 4 listes, chacune associé à un bord, dont l'indexation est définie comme suit :
* 0 : bord gauche
* 1 : bord haut
* 2 : bord droit
* 3 : bord bas
Chacune de ces listes contient à la fin 1 ou 2 éléments. S'il n'y en a qu'un, le côté associé est suffisamment éloigné de notre point. S'il y en a 2, alors notre point est trop proche du côté concerné i et se trouve à une distance stockée dans le deuxième élément.

Si le point (x0, y0) se trouve sur le bord gauche pas pas dans un coin, alors il n'y a qu'un problème : on exécute alors `resolution_1_probleme_initial`. Les arguments `x1` et `x2` sont les abscisses des bords gauche et droit, dans le cas normal elles valent `x0+delta` et `x0-delta`. Idem pour `y1` et `y2` pour les côtés bas et haut.

Si le point (x0, y0) se trouve dans un coin, il y a 2 problèmes : on exécute alors `resolution_2_problemes_initial`.

Simple_contour_carre_initial ?????

Maintenant, on lance la recherche de tous les autres points tant que les conditions d'arrêt ne sont pas respectées.
On note (x0, y0) le point courant. On commence par regarder les bords dont notre point est trop proche avec `test_bord` et on compte combien de problèmes (0, 1 ou 2) on a recontré avec `nombreprobleme`.

**Si le nombre de problèmes vaut 0,** alors on exécute la fonction qui s'appliquera le plus souvent : `simple_contour_carre`. Elle cherche le point suivant sur les quatre côtés du petit carré, en terminant par le côté d'où l'on vient. Il est peut-être nécessaire de détailler ici notre raisonnement avec la variable `numcôté`.\
`numcôté` vaut toujours O, 1, 2 ou 3. Elle représente, du point de vue du point précédent (xx, yy), ie celui à partir duquel on a trouvé (x0, y0), le numéro du côté sur lequel se trouve (x0, y0) avec la numérotation donnée plus haut. Prenons un exemple : si (x0, y0) a été trouvé sur le côté gauche de (xx, yy), alors `numcôté=0`.\
Il s'agit maintenant d'adapter le sens de `numcôté` au point courant (x0, y0). Dans notre exemple, (xx, yy) se trouve, du point de vue du point courant (x0, y0), sur le bord droit ! C'est donc le côté 2 qu'il faut regarder en dernier, d'où la liste `L_côté = [3, 0, 1, 2]`.\
\
Ensuite, on cherche le point suivant sur les 3 premiers côtés de L_côté.
\
Si cependant on ne trouve pas de points, on est obligé de regarder sur le dernier côté, ie le côté sur lequel se trouve (xx, yy). C'est un cas de rebroussement. On traite ce cas séparément car il ne faudrait pas retrouver (xx, yy). On fait alors appel à un find_seed spécial : `find_seed_except`.\
`find_seed_except` exécute deux fois la fonction `find_seed` (à x0 ou y0 fixé, peu importe) : 
1. Sur un premier intervalle de `valmin` jusqu'à `xx-eps` ou `yy-eps` (on rappelle que nos points sont tous déterminés à `eps` près).
2. Sur un deuxième intervalle de `xx+eps` ou `yy+eps` jusqu'à `valmax`.
L'idée est donc d'exclure une petite zone autour de (xx, yy), de longueur `2*eps`, sur laquelle on ne cherche pas le point suivant.\
\
*Remarque* : Si delta est pris suffisamment petit, on ne devrait jamais avoir à exécuter `find_seed_except`, car un rebroussement de la ligne de niveau à si petite échelle est très peu probable si l'on prend une fonction f pas trop exotique...

In [ ]:
#simple_contour_carre et find_seed_except.

**Si le nombre de problème vaut 1,** alors on exécute `resolution_1_probleme`. L'idée est la même que dans `resolution_1_probleme_initial`, on traite seulement le cas de rebroussement en plus, de la même façon que dans `simple_contour_carre`.

**Si le nombre de problèmes vaut 2,** alors on exécute `resolution_2_problemes`. L'idée est d'exécuter `resolution_1_probleme`en ayant réajuster un des côtés du carré, de sorte qu'on se ramène à un cas où il n'y a plus que 1 problème. On n'oublie pas au passage de spécifier qu'il n'y a plus de problème sur le côté qu'on vient de modifier dans la liste `problème`.

Il ne nous reste plus qu'à construire la structure globale de notre fonction `contour_simple` en appelant toutes les fonctions précédentes.

#### La fonction contour_complexe